# CSE6250BDH Deep Learning Labs
## 0. Introduction to PyTorch

In this chapter, we will learn basic usage of PyTorch.
There are many good tutorials on PyTorch on web.
We highly recommend you to follow the official [tutorial](http://pytorch.org/tutorials/) even though this tutorial is also mainly from it.

### Import

After installing PyTorch, you can import `torch` in Python to use PyTorch.

In [1]:
import torch

### Tensor Creation

PyTorch is very similar with Numpy as they say it is a replacement for Numpy to use the power of GPUs. Although there are still missing components, it has many same/similar functions for constructing or manipulating 'Tensor's.

A basic object used in PyTorch is 'Tensor' which is equivalent to 'ndarray' in Numpy. Similarly to Numpy, there are multiple types of Tensors, e.g. Float, Double, Int, Long, etc. Most of time, however, we will use FloatTensor mainly (and it is a default type for the most of functions) to utilize GPU and LongTensor sometime for target/label values.

Lets try to create a Tensor. If you call `torch.Tensor(rows, cols)`, it will return a FloatTensor without initialization (with garbage values).

In [2]:
x = torch.Tensor(5, 3) # same result with torch.FloatTensor(5,3)
x


 2.8887e-06  4.5590e-41  2.8887e-06
 4.5590e-41         nan  4.5590e-41
 4.4721e+21  1.6647e-41  6.7262e-44
 0.0000e+00  6.7262e-44  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00
[torch.FloatTensor of size 5x3]

You can create initialized Tensor filled with 1s, 0s, or random numbers from a uniform distribtution by using `torch.ones`, `torch.zeros`, or `torch.rand` repectively.

In [3]:
x_ones = torch.ones(5,3)
print(x_ones)

x_zeros = torch.zeros(5,3)
print(x_zeros)

x_uniform = torch.rand(5,3)
print(x_uniform)


 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
[torch.FloatTensor of size 5x3]


 0  0  0
 0  0  0
 0  0  0
 0  0  0
 0  0  0
[torch.FloatTensor of size 5x3]


 0.9569  0.2294  0.4642
 0.8916  0.0795  0.8181
 0.7395  0.1194  0.9653
 0.0053  0.9745  0.8530
 0.1947  0.5780  0.4946
[torch.FloatTensor of size 5x3]



### Exercise: Try `torch.eye`, `torch.linspace`, `torch.logspace`, etc.
### Exercise: Try other random functions from [here](http://pytorch.org/docs/master/torch.html#random-sampling)

### Converting from/to Numpy ndarray

You can also create a Tensor from Numpy ndarray or vice versa. In fact, we may do this many times in a project since we want to utilize many Numpy-based libraries (e.g., Pandas, Scikit-learn, Matplotlib, etc.) as well as GPU computation.

You can simply call `torch.from_numpy(ndarray)` to create a `Tensor` from a `numpy.ndarray`. **Be careful that the returned Tensor and original ndarray share the same memory**. Therefore, if you modify the Tensor, it will be reflected in the ndarray.

In [4]:
import numpy as np
np_array = np.array([1., 2., 3.])
print(np_array)
torch_tensor = torch.from_numpy(np_array)
print(torch_tensor)

# Modify the Tensor
torch_tensor[0] = -1.0
print(np_array)

[ 1.  2.  3.]

 1
 2
 3
[torch.DoubleTensor of size 3]

[-1.  2.  3.]


For the reverse way of conversion, you can call `numpy()` on a Tensor. Again, resulting ndarray shares the memory with the Tensor.

In [5]:
another_torch_tensor = torch.rand(3)
print(another_torch_tensor)
another_np_array = another_torch_tensor.numpy()
print(another_np_array)

# Modify ndarray
another_np_array[0] *= 2.0
print(another_torch_tensor)


 0.0762
 0.2101
 0.8670
[torch.FloatTensor of size 3]

[ 0.07623515  0.21010938  0.86702114]

 0.1525
 0.2101
 0.8670
[torch.FloatTensor of size 3]



### Basic Operations

#### Indexing

You can use standard numpy-like indexing.

In [6]:
A = torch.rand(3,3)
print(A)
print(A[:, 1])
print(A[:2, :])


 0.4033  0.9692  0.4608
 0.6637  0.4313  0.4555
 0.2906  0.6024  0.3688
[torch.FloatTensor of size 3x3]


 0.9692
 0.4313
 0.6024
[torch.FloatTensor of size 3]


 0.4033  0.9692  0.4608
 0.6637  0.4313  0.4555
[torch.FloatTensor of size 2x3]



#### Arithmetic Operations
Arithmetic operations with `+-*/` operators are all element-wise computation. Therefore, if you want to do some matrix computations such as matrix-matrix (or vector) multiplication, you need to call separate functions.  

In [7]:
B = torch.rand(3,3)
print(A+B)
print(A*B)
# Another elementwise multiplication
print(torch.mul(A,B))

# Matrix-Matrix multiplication
print(torch.mm(A,B))
# Matrix-Vector multiplication
print(torch.mv(A,B[:,1]))


 0.4036  1.1821  0.6642
 0.6681  1.0320  1.4105
 1.1630  1.3094  0.9687
[torch.FloatTensor of size 3x3]


 0.0001  0.2063  0.0937
 0.0029  0.2591  0.4350
 0.2535  0.4259  0.2212
[torch.FloatTensor of size 3x3]


 0.0001  0.2063  0.0937
 0.0029  0.2591  0.4350
 0.2535  0.4259  0.2212
[torch.FloatTensor of size 3x3]


 0.4064  0.9939  1.2840
 0.3995  0.7224  0.8201
 0.3245  0.6845  0.8557
[torch.FloatTensor of size 3x3]


 0.9939
 0.7224
 0.6845
[torch.FloatTensor of size 3]



There are many predefined operations for your convenience such as batch multiplication with addition, etc. Please read [PyTorch Docs](http://pytorch.org/docs/master/torch.html#math-operations) for more information.

### GPU Acceleration

If we have NVIDIA GPU(s), we can accelerate computation once we move Tensors onto GPU.
Let's compare how much GPU can accelerate especially matrix operations.
We will do a matrix-matrix multiplication between two 5k-by-5k matrices on both CPU and GPU.

In [8]:
mat_cpu = torch.rand(5000, 5000)
mat_cpu


 2.9307e-01  5.2394e-01  1.6733e-01  ...   7.5681e-01  8.7937e-01  5.9640e-01
 5.6545e-01  9.0258e-01  2.8607e-01  ...   9.1151e-01  2.0318e-01  5.2510e-01
 2.8487e-01  6.8594e-01  9.5869e-01  ...   9.8145e-01  3.2971e-01  6.7894e-03
                ...                   ⋱                   ...                
 3.2276e-01  9.4729e-01  4.4242e-01  ...   3.7736e-01  2.7107e-01  9.8560e-01
 2.8132e-01  1.8895e-01  9.6733e-01  ...   5.5537e-01  8.5894e-01  9.1208e-01
 9.2259e-01  8.2838e-01  1.9059e-01  ...   5.8640e-02  6.2859e-01  3.3649e-01
[torch.FloatTensor of size 5000x5000]

In [9]:
%%time
torch.mm(mat_cpu.t(), mat_cpu)

CPU times: user 3.13 s, sys: 336 ms, total: 3.47 s
Wall time: 592 ms



 1692.1068  1268.0529  1269.9794  ...   1253.7578  1243.3914  1241.6606
 1268.0529  1670.3572  1270.6412  ...   1263.1759  1238.3883  1249.3873
 1269.9794  1270.6412  1682.7657  ...   1263.9650  1253.0474  1262.2773
              ...                  ⋱                 ...               
 1253.7577  1263.1757  1263.9650  ...   1675.8093  1238.7645  1243.4160
 1243.3914  1238.3884  1253.0475  ...   1238.7645  1639.2242  1233.6138
 1241.6608  1249.3873  1262.2773  ...   1243.4160  1233.6138  1662.8052
[torch.FloatTensor of size 5000x5000]

#### We need a GPU for this comparison
We can check its availability like:

In [10]:
if torch.cuda.is_available():
    cuda = True
else:
    cuda = False
cuda

True

In [11]:
mat_gpu = torch.rand(5000, 5000)
if cuda:
    mat_gpu = mat_gpu.cuda()
mat_gpu


 1.2666e-01  8.2122e-01  2.0375e-01  ...   1.4205e-01  6.0894e-01  2.4421e-01
 7.0849e-01  7.8901e-01  6.9667e-01  ...   4.2608e-01  8.3568e-01  8.9200e-01
 4.7030e-01  6.2742e-01  5.3437e-01  ...   3.4233e-01  8.5170e-01  4.2634e-01
                ...                   ⋱                   ...                
 1.2858e-01  2.2800e-01  6.6128e-01  ...   2.7187e-01  9.0233e-01  9.9726e-01
 8.4991e-02  6.3336e-01  5.6319e-01  ...   1.1971e-01  5.9710e-01  6.0663e-01
 3.8146e-01  4.8843e-01  6.6677e-02  ...   6.1802e-01  6.2672e-01  9.3424e-01
[torch.cuda.FloatTensor of size 5000x5000 (GPU 0)]

In [12]:
%%time
torch.mm(mat_gpu.t(), mat_gpu)

CPU times: user 84.8 ms, sys: 12.1 ms, total: 96.9 ms
Wall time: 96.4 ms



 1665.6709  1238.1910  1259.5555  ...   1255.5841  1231.5492  1227.7648
 1238.1910  1673.3875  1255.0764  ...   1240.6320  1243.0193  1229.4028
 1259.5555  1255.0764  1674.4164  ...   1253.9111  1228.9789  1240.1782
              ...                  ⋱                 ...               
 1255.5841  1240.6320  1253.9111  ...   1677.7839  1227.0667  1244.6802
 1231.5492  1243.0193  1228.9789  ...   1227.0667  1622.8091  1225.3038
 1227.7648  1229.4028  1240.1782  ...   1244.6802  1225.3038  1644.8094
[torch.cuda.FloatTensor of size 5000x5000 (GPU 0)]

Can you see the speed-up? It will be much critical if we use larger matrices, more matrix computations, and a deeper neural network model.

### Variable
PyTorch provide a functionality of automatic differentiation with a package `autograd` and Variable is the key class for utilizing it.

Variable wraps a Tensor as its data and maintain another Tensor for the gradient with respect to this data Tensor. Also, almost all of built-in operations in PyTorch supports automatic differentiation with Variable. Therefore, we can call `.backward()` on a computation graph, e.g. neural network, after we finish our computation on the graph, then we can get automatically accumulated gradient for each Variable related with the graph.

Let's try a simple example for easier understanding.

In [13]:
from torch.autograd import Variable

# Create some Tensors and a Variable
x = Variable(torch.FloatTensor([2.0]), requires_grad=False)
w = Variable(torch.FloatTensor([0.5]), requires_grad=True)
b = Variable(torch.FloatTensor([0.1]), requires_grad=True)
print(x)
print(w)
print(b)

# Define a computational graph
y = w*x + b # Currently, y = 0.5x + 0.1 and y(2) = 1.1
print(y)

Variable containing:
 2
[torch.FloatTensor of size 1]

Variable containing:
 0.5000
[torch.FloatTensor of size 1]

Variable containing:
 0.1000
[torch.FloatTensor of size 1]

Variable containing:
 1.1000
[torch.FloatTensor of size 1]



Let's compute gradients on the graph y and print the gradient w.r.t each Variable.

In [14]:
# Compute gradients
y.backward()

print(x.grad)
print(w.grad)
print(b.grad)

None
Variable containing:
 2
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]



Since we set `requires_grad=False` for Variable `x`, it has `None` value.
Also, if we do a simple math to differentiate it manually, we can easily get:
$$
\frac{\partial y}{\partial w} = \frac{\partial}{\partial w}\left(wx + b\right) = x\\
\text{and}\\
\displaystyle \frac{\partial y}{\partial w}\Bigr|_{x=2} = 2 
$$
Similarly,
$$
\frac{\partial y}{\partial b} = \frac{\partial}{\partial b}\left(wx + b\right) = 1\\
\text{and}\\
\displaystyle \frac{\partial y}{\partial b}\Bigr|_{x=2} = 1 
$$

Thanks to the functionality of automatic differentiation, we can build a very complex computational graph such as a neural network with many layers without manually computing the gradients of parameters.

Please refer to the official [tutorial](http://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html) for more details.

In the next chapter, we will build a simple feed-forward neural network by using these components of PyTorch we have learnt.